In [1]:
from utils.file_reader import read_file
from utils.search_capability import *

In [22]:
from datasets import load_dataset

# Path to your GAIA loader script
loader_path = "./GAIA.py"

# Levels to load
levels = ["2023_level1"]

# Load datasets for each level
datasets = {}
for level in levels:
    datasets[level] = load_dataset(loader_path, name=level, split="validation")
    
    # Print all examples in the dataset
    print(f"\n=== All examples from {level} ===")
    print(f"Total examples: {len(datasets[level])}")
    
    for i, example in enumerate(datasets[level]):
        print(f"\n--- Example {i+1} ---")
        if i == 0:
            for k, v in example.items():
                # For large values like annotator metadata, print just a summary
                if isinstance(v, dict):
                    print(f"{k}: {type(v)} with keys {list(v.keys())}")
                
                else:
                    print(f"{k}: {v}")
            break



=== All examples from 2023_level1 ===
Total examples: 53

--- Example 1 ---
task_id: e1fc63a2-da7a-432f-be78-7c4a95598703
Question: If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.
Level: 1
Final answer: 17
file_name: 
file_path: 
Annotator Metadata: <class 'dict'> with keys ['Steps', 'Number of steps', 'How long did this take?', 'Tools', 'Number of tools']


In [3]:
example["Question"]

"An office held a Secret Santa gift exchange where each of its twelve employees was assigned one other employee in the group to present with a gift. Each employee filled out a profile including three likes or hobbies. On the day of the gift exchange, only eleven gifts were given, each one specific to one of the recipient's interests. Based on the information in the document, who did not give a gift?"

In [4]:
example["Annotator Metadata"]

{'Steps': '1. Open the document.\n2. Look at gifts and recipient interests.\n3. Match Galileo Galilei biography (could apply to astronomy or books -> Miguel or Micah)\n4. Match fishing reel (only applies to fishing -> Harry)\n5. Match Raku programming guide (Perl language, but could also apply to JavaScript enthusiast - > Fred or Jun)\n6. Match chisel set (could apply to camping or woodworking, but Harry is already fulfilled -> Jun, so Raku guide is for Fred)\n7. Match custom dice (could apply to board games or tabletop RPGs -> Lucy or Sara)\n8. Match “War and Peace” American film copy (could apply to old movies or Audrey Hepburn -> Perry or Alex)\n9. Match yarn (only applies to knitting -> Micah, so the Galileo biography is for Miguel)\n10. Match "One Piece" graphic novel (could apply to books or manga, but Micah already has yarn -> Alex, so the "War and Peace" film is for Perry)\n11. Match "War and Peace" novel (could apply to books or historical fiction novels, but Micah has yarn ->

In [5]:
read_file(example["file_path"])

'Employees\n\nHarry\nRebecca\nGeorgette\nMicah\nPerry\nTyson\nLucy\nJun\nSara\n Miguel\n Fred\n Alex\n\nGift Assignments\n\n\n\nProfiles\n\nHarry: Fishing, Camping, Wine\nRebecca: Cars, Dogs, Chocolate\nGeorgette: Yoga, Cooking, Green Energy\nMicah: Knitting, Rainy Weather, Books\nPerry: Old Movies, Rats, Journaling\nTyson: Historical Fiction Novels, Biking, Parakeets\nLucy: Coffee, Physics, Board Games \nJun: Woodworking, Barbecue, JavaScript\nSara: Tabletop RPGs, Spas, Music\nMiguel: Astronomy, Decorative Washi Tape, Ketchup\nFred: Chemistry, Perl, Cats\nAlex: Surfing, Audrey Hepburn, Manga\n\nGifts:\n\nGalileo Galilei biography\nFishing reel\nRaku programming guide\nChisel set\nCustom dice\n“War and Peace” American film copy\nYarn\n“One Piece” graphic novel\n“War and Peace” novel\nStarbucks gift card\nFoam exercise mat'

In [10]:
import ollama

query = example["Question"] + "INFO:" + read_file(example["file_path"])

response = ollama.chat(
    model='deepseek-r1:32b',
    messages=[
        {'role': 'user', 'content': query}
    ]
)

print(response['message']['content'])

<think>
Okay, so I'm trying to figure out who didn't give a gift in the Secret Santa exchange. There are twelve employees, but only eleven gifts were given. That means one person didn't give a gift, probably because they weren't assigned someone or maybe their assignment didn't result in a gift. 

First, let me list all the employees: Harry, Rebecca, Georgette, Micah, Perry, Tyson, Lucy, Jun, Sara, Miguel, Fred, Alex. That's twelve people.

Now, each person was assigned to give a gift to another, so there should be twelve assignments, but only eleven gifts were given. So one person didn't end up giving a gift because maybe they were the one who wasn't assigned? Or perhaps someone gave a gift but it wasn't listed, but the problem says only eleven gifts were given, each specific to the recipient's interests.

So I need to figure out which employee didn't give a gift. The key is that each gift corresponds to one of the recipient's likes or hobbies. So I should match each gift to an employ

In [21]:
query = "## TASK: Convert the following TEXT to JSON with the key answer in it. ## TEXT:" + response['message']['content'] + "## ANSWER:"

response1 = ollama.chat(
    model='llama3:8b',
    messages=[
        {'role': 'user', 'content': query}
    ]
)

print(response1['message']['content'])

Here is the TEXT converted to JSON with the key "answer" containing the value:

```json
{
  "text": "<think>\n## TASK: Convert the following TEXT to JSON with the key answer in it. ##\nTEXT:<think>\nThe employee who did not give any gift is likely Fred. His unique interests in technical subjects may have made it difficult for others to find an appropriate gift, leading him to not participate.\nAnswer: \boxed{Fred}\n</think>",
  "answer": "Fred"
}
```


### Search based question

In [24]:
example["Question"]

'If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.'

In [27]:
import ollama

query = "### QUESTION:" + example["Question"]+ "WHAT FACTS DO WE NEED TO ACCESS FROM THE INTERNET TO CORRECTLY ANSWER THIS QUESTION. PROVIDE A LIST OF UPTO 10 QUERIES"

response = ollama.chat(
    model='llama3:8b',
    messages=[
        {'role': 'user', 'content': query}
    ]
)
print(response['message']['content'])

What a fascinating question!

To answer this, we'll need to gather some facts from the internet. Here are the queries:

1. **Eliud Kipchoge's marathon pace**: What is Eliud Kipchoge's average speed in kilometers per hour for his record-breaking marathons?
2. **Earth-Moon distance**: What is the minimum perigee value (i.e., closest approach) of the Moon to Earth, measured in kilometers or meters?
3. **Marathon distance**: How long is a standard marathon race, in kilometers or miles?
4. **Kipchoge's average speed conversion**: If Kipchoge's pace is given in kilometers per hour, can we convert it to miles per hour or vice versa for easier calculations?
5. **Time calculation unit**: Should we perform the time calculation in hours, minutes, or seconds? (Assuming we'll round to the nearest thousand hours)
6. **Moon's orbital speed**: Is there a reliable estimate of the Moon's average orbital speed around Earth, which could help us compare Kipchoge's pace?
7. **Kipchoge's endurance limit**: I

In [30]:
query1 = "Conver the following to Python list:" + response['message']['content'] + "### LIST:" 
response1 = ollama.chat(
    model='llama3:8b',
    messages=[
        {'role': 'user', 'content': query1}
    ]
)
print(response1['message']['content'])

Here is the list converted to Python:

```python
questions = [
    "Eliud Kipchoge's marathon pace",
    "Earth-Moon distance",
    "Marathon distance",
    "Kipchoge's average speed conversion",
    "Time calculation unit",
    "Moon's orbital speed",
    "Kipchoge's endurance limit",
    "Speed conversion",
    "Earth-Moon distance variation",
    "Rounding rules"
]
```

Let me know if you'd like me to help with any of these questions or if there's anything else I can assist you with!


In [1]:
questions = [
    "Eliud Kipchoge's marathon pace",
    "Earth-Moon distance",
    "Marathon distance",
    "Kipchoge's average speed conversion",
    "Time calculation unit",
    "Moon's orbital speed",
    "Kipchoge's endurance limit",
    "Speed conversion",
    "Earth-Moon distance variation",
    "Rounding rules"
]
process_questions(questions)

NameError: name 'process_questions' is not defined